In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [3]:
root_folder = "./tactile_dataset/"
file_name = "final_merged_df_sw500.csv"

df = pd.read_csv(root_folder + file_name)

In [4]:
X = df[['baro','imu_ax', 'imu_ay', 'imu_az','imu_gx', 'imu_gy', 'imu_gz','imu_mx', 'imu_my', 'imu_mz']].values
y = df['Texture']

print(f"Total samples collected: {X.shape[0]}")
print(f"Feature vector size: {X.shape[1]}")
print(f"Unique textures: {np.unique(y)}")

Total samples collected: 90665
Feature vector size: 10
Unique textures: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
texture_classes = label_encoder.classes_
num_classes = len(texture_classes)
print("\nEncoded texture labels:", texture_classes)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)


Encoded texture labels: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [46]:
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)

class ANN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.softmax(out)
        return out

In [47]:
model = ANN(10, 100, 12)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [48]:
num_epochs = 20

for epoch in range(num_epochs):
    accuracy = 0
    for i, (X_batch, y_batch) in enumerate(train_loader):
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        accuracy += (y_batch == y_pred.argmax(1)).sum().item()
    print(f"Epoch {epoch+1}, loss: {loss.item()}, accuracy: {accuracy/X_train.shape[0]}")

Epoch 1, loss: 2.210732936859131, accuracy: 0.4103843820658468
Epoch 2, loss: 2.3162314891815186, accuracy: 0.44897424585010753
Epoch 3, loss: 2.121678113937378, accuracy: 0.4615617934153201
Epoch 4, loss: 2.2181084156036377, accuracy: 0.47016489273699885
Epoch 5, loss: 2.362545967102051, accuracy: 0.4610792477802901
Epoch 6, loss: 2.0344324111938477, accuracy: 0.46688358241879446
Epoch 7, loss: 2.2655422687530518, accuracy: 0.4681106270335852
Epoch 8, loss: 2.0187222957611084, accuracy: 0.4666354160922076
Epoch 9, loss: 2.180729389190674, accuracy: 0.45560580157723485
Epoch 10, loss: 2.1185925006866455, accuracy: 0.450408095737054
Epoch 11, loss: 2.168163299560547, accuracy: 0.4539651464181327
Epoch 12, loss: 2.1688053607940674, accuracy: 0.43284343462195995
Epoch 13, loss: 2.1686606407165527, accuracy: 0.4508079192632217
Epoch 14, loss: 2.1796202659606934, accuracy: 0.4397369436938179
Epoch 15, loss: 2.0675644874572754, accuracy: 0.43751723377267965
Epoch 16, loss: 2.2942891120910645